---
title: "Informative Priors for Marketing Mix Modeling (MMM) Calibration in PyMC-Marketing"
---

Let's talk about a puzzle that many marketers face — understanding the incremental impact of their campaigns. 

The quest to answer this question gave rise to cookies, pixels, and various user-level identifiers on the web and digital applications. Now, as these tools fade away due to heightened virtual security measures, this drive has sparked the development of new statistical models to find the answer.

Marketing Mix Modeling (MMM) models, along with randomized and quasi-experiments, have emerged as key tools. Yet, measuring how advertisements influence people's decision-making is inherently challenging. Often, the methodology or model you choose and the conditions in which you apply it can lead to varying interpretations of reality.

**How can we align models that measure various aspects of our multifaceted phenomenon?** Can we consolidate all our previous knowledge and experience from different fields into a single methodology?

This brings us to the concept of **calibration** and how to apply it within the Bayesian methodology.

# Finding priors through quasi-experiments

*Imagine this*: your company decides to dial back its advertising budget in **Venezuela** between March 28th and May 5th. Now, as a marketer, you're scratching your head, wondering: Did this decision affect sales? If so, how significantly?

Before diving into complex techniques, let's consider how to measure this specific action! You can unravel this mystery through the power of experiments! 🔬

Although your company reduced spending in **Venezuela**, it maintained its usual advertising efforts in other countries. This scenario provides a unique chance to explore an alternate reality—what would have happened if the spending hadn't been cut?

Let's pick a country—say, **Colombia**—where your advertising efforts remained unchanged during the same period. *Colombia* will be your control group, providing a baseline to compare against.

But why could we use this country as a comparison? This approach is grounded in the hypothesis that *Venezuela* and *Colombia* might be exposed to similar factors and conditions to a certain extent. By treating *Venezuela*'s sales as a function where *Colombia's* sales are a factor—owing to their exposure to similar conditions—we can account for much of the variability. This method allows us to focus on analyzing the residuals to see if they show any significant variations during the treatment period, offering deeper insights into the effects of our strategic decisions.

$$ 
    \text{Venezuela Sales} = \text{Colombia Sales} \cdot \beta + \text{Venezuela Exogenous Variables}
$$

If we do this, we'll be assuming a relationship presented in the following DAG:

In [ ]:
#!collapse
import graphviz
from IPython.display import display
import io
from PIL import Image

# Create a new directed graph with the updated relationships
dot = graphviz.Digraph()

# Adding nodes with specific ranks
with dot.subgraph() as s:
    s.attr(rank='same')
    s.node('weather')
    s.node('marketing')
    s.node('Seasonality')
    s.node('Colombia Exogenous variables')
    s.node('Venezuela Exogenous variables')

dot.node('Colombia Sales')
dot.node('Venezuela Sales')

# Adding edges based on the new relationships
edges = [
    ("weather", "Colombia Sales"),
    ("weather", "Venezuela Sales"),
    ("marketing", "Colombia Sales"),
    ("marketing", "Venezuela Sales"),
    ("Seasonality", "Colombia Sales"),
    ("Seasonality", "Venezuela Sales"),
    ("Colombia Exogenous variables", "Colombia Sales"),
    ("Venezuela Exogenous variables", "Venezuela Sales")
]

for edge in edges:
    dot.edge(edge[0], edge[1])

# Define styles for specific nodes
dot.attr('node', shape='circle', style='filled', color='white', fontcolor='black')
dot.attr('edge', color='black')

# Set color to grey for specific nodes
grey_nodes = ["Colombia Sales", "Venezuela Sales"]
for node in grey_nodes:
    dot.node(node, style='filled', color='grey')

# Render the graph to a PNG image in a byte stream
byte_stream = io.BytesIO(dot.pipe(format='png'))

# Load the image from the byte stream and display it
image = Image.open(byte_stream)
display(image)

**Although this selection remains somewhat speculative**, it's a reasonable choice (At least for the example). Both countries are in the north of South America, located near the Caribbean, and share similar climates. Additionally, their cultures are closely related, having been part of the same country less than two centuries ago. We can say they are representatively similar.

> **Note A**: This methodology isn't limited to countries; it's just as applicable to regions or cities, as long as you have consistent assumptions that can be proven by data. Don't forget that other time series should not be affected by changes in advertising (or the action being measured).


> **Note B**: Let's observe that the DAG does not consider spill-over effects. For example, when evaluating an ad campaign in one city, you have to be careful not to include neighboring cities because different people can travel and see the ads. We are not considering those cases here.

Now, let's review why we are doing all this? Why are we assuming these conditions? Let's imagine creating two realities for the target variable (*Venezuelan* sales): one, the real world where you cut advertising (the "*Treatment*" group) and second, a hypothetical scenario where everything stayed the same (the "*Control*" group, generated by *Colombia*). By comparing these two versions, you can directly measure what might have been lost or gained by stopping marketing in Venezuela.

As we mentioned earlier, by observing these differences, you gain insights into the impact of your decisions but also empower yourself to make more informed choices in the future. Isn’t that a compelling way to approach marketing strategy? If your answer is positive, you are in the right place, and therefore, it is time to introduce yourself to your new best friend to solve these riddles: [CausalPy](https://causalpy.readthedocs.io/en/latest/).

I'm sure you're wondering, what is **CausalPy**? It's a library from the PyMC ecosystem, designed to facilitate causal inference analyses. Using [CausalPy](https://causalpy.readthedocs.io/en/latest/) we can explore different quasi-experimental methods to causally measure the effect of our actions, in the absence of natural or randomized experiments.

That being said, it's time to jump into action 🔥

## Considerations
Before I forget it, we have to go over a few things.

## 1. Prerequisite Knowledge

The notebook assumes the reader knows the essential functionalities of PyMC-Marketing. If one is unfamiliar, the ["Marketing Mix Modeling (MMM) Example Notebook"](https://www.pymc-marketing.io/en/stable/notebooks/mmm/mmm_example.html) serves as an excellent starting point, offering a comprehensive introduction to Marketing Mix Modelings in this context.

## 2. Installing PyMC-Marketing

Before delving into the specifics of experiments and priors, the initial step is to install the **PyMC-Marketing** and **CausalPy** libraries and ascertain its version. 

The following command can be run on your Jupyter Notebook:
> `conda install pymc-marketing causalpy`

- Install instructions for PyMC-Marketing: [here](https://www.pymc-marketing.io/en/stable/installation.html)
- Install instructions for CausalPy: [here](https://causalpy.readthedocs.io/en/latest/installation.html)

If everything went well now we are ready to start 🚀

## Importing libraries

In [ ]:
#!collapse
import warnings

import arviz as az
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pymc_marketing.mmm import MMM

import causalpy as cp
import pymc as pm

### Notebook configuration ⚙️
The following code is a small configuration for the notebook, to make it more readable and to avoid warnings. Defining a pre-defined style.

In [ ]:
#!collapse
warnings.filterwarnings("ignore")

az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

### Seed specification
The following code generates a seed to ensure reproducibility of the results. This seed will be used throughout the notebook to maintain consistency in the analysis.

In [ ]:
#!collapse
seed: int = sum(map(ord, "Calibration through Priors is amazing!!"))
rng: np.random.Generator = np.random.default_rng(seed=seed)

# Understanding our dataset 👀

The following dataset contains everything we need to train a Marketing Mix Modeling, and conduct our experiment. 
- The columns for `Venezuela` and `Colombia` represent sales in each country. 
- The columns with the names of **digital channels** represent the impressions or advertising spend on each channel. 
- The `ds` column represents the date of the data, currently at weekly granularity.

In [ ]:
df = pd.read_csv("./datasets/dataset_chapter_n.csv", index_col=0).dropna()[["ds", "venezuela", "colombia", "meta", "google", "pinterest", "trend"]]
df["ds"] = pd.to_datetime(df["ds"])
df.info()

## Observing the sales 💶

Let's pause for a moment and give our dataset a quick once-over before we proceed. We need to assess how well our sales fared in all countries before we apply any changes to just one of them. 

Bear in mind that the more similar the countries behave, the more accurate our counterfactual estimate will be.

In [ ]:
#!collapse
start_date = pd.to_datetime("2022-03-28") #Action start date
end_date = pd.to_datetime("2022-05-20") #Action end date

fig, ax = plt.subplots(
    nrows=2, ncols=1, figsize=(10, 7), sharex=True, sharey=True, layout="constrained"
)
sns.lineplot(x="ds", y="venezuela", data=df, color="C0", ax=ax[0])
sns.lineplot(x="ds", y="colombia", data=df, color="C1", ax=ax[1])
ax[0].axvline(start_date, color="black", linestyle="--", alpha=0.5)
ax[0].axvline(end_date, color="black", linestyle="--", alpha=0.5)
ax[1].axvline(start_date, color="black", linestyle="--", alpha=0.5)
ax[1].axvline(end_date, color="black", linestyle="--", alpha=0.5)
ax[1].set(xlabel="date")
fig.suptitle("Sales on Venezuela v/s Colombia over time", fontsize=16)
plt.show()

Notice how the sales from *Venezuela* and *Colombia* were in sync until just before the intervention period, marked by the vertical black lines, where *Venezuela* appeared to experience a decline.

> Note: This visual check can validate our hypothesis (DAG) that both countries share common factors, explaining why one performs well in estimating the other.

Taking a simple look, we can notice that something has happened. But how can we quantify it?

# **CausalPy** to the rescue 🚀

Let’s dive into the heart of the process using a nifty tool in `CausalPy`, the `pymc_experiments.InterruptedTimeSeries` class. Think of it as your personal time travel machine, ready to dissect exactly when and how your strategies altered the course of your sales trajectory. 

The `InterruptedTimeSeries` class requires the following parameters:

- `data`: The data used to fit the model.
- `treatment_time`: The time at which the treatment was applied.
- `formula`: The formula used to fit the model.
- `model`: A forecasting model trained on the data.

Once our model is humming and all parameters are in place, we can observe the **estimated effect**. The estimated effect is the difference between the observed variable and the counterfactual variable. It represents the impact of the action (treatment) on the target variable.

In [ ]:
experiment = cp.pymc_experiments.InterruptedTimeSeries(
    data=df.query(f"ds<='{end_date}'").set_index("ds"),
    treatment_time=start_date,
    formula="venezuela ~ colombia",
    model=cp.pymc_models.LinearRegression(sample_kwargs={"random_seed": seed, "target_accept": 0.95, "tune": 2000}),
)

fig, ax = experiment.plot()
plt.xticks(rotation=45, fontsize=8)
plt.show()

Let's take a look at the output. We can see a figure with 3 subplots:

1. The first subplot depicts the fit of our model before and during the intervention. It is evident from the image that prior to the intervention, the model was accurately predicting the sales in Venezuela based on the sales in Colombia. However, after the intervention, there was a noticeable divergence, leading to the appearance of a blueish delta in the figure.

2. The subsequent image provides us with another angle of the same data. We can discern that the difference between predictions and actual sales before the intervention was zero, but after the intervention, it increased up to `-1.33`. This implies that the relationship between variables underwent a significant change after the intervention.

3. the third image portrays how the daily differences that we observed in the second image keep accumulating over time, leading to a large total effect.

As you can see, these three images allow us to reveal and quantify the impact of our actions. But I'm sure you're wondering, how can we know that these differences are actually the causal effect of our actions and not simply an error in the model estimations?

## **Quasi-Experiments considerations** ⚠️

To pinpoint the cause of any changes observed in a time series, it's super important to make sure that the model effectively controls all the factors that may impact it. The dataset used in this case was created synthetically (based on the previous DAG exposed), which made the job easier by using only "Colombia" as a predictor to capture the effect. But in real-world situations, you gotta include additional variables that account for all the different conditions that might arise during an experiment. If you don't, you might mistakenly attribute an effect to an action where there's no connection whatsoever.

When conducting a quasi-experiment, it's crucial not only to control for the right variables but also to **uphold the assumption that no other explanations could account for the results**. Failing to do so compromises the experiment's reliability. While we can never control every factor, if there's another reason for a change to occur, how can we confidently attribute it to our action? Ensuring the validity of our results hinges on this careful consideration.

To determine if your causal model is well calibrated before the experiment, here is some recommended reading.:
1. [Bayesian Power Analysis in CausalPy](https://github.com/pymc-labs/CausalPy/issues/276)
2. [A/A Test with PyMC](https://juanitorduz.github.io/time_based_regression_pymc/)

A few other important consideritions could be:
1. [Cross-validation analysis](https://juanitorduz.github.io/time_based_regression_pymc/)

## Estimating the total causal effect 🔬

We have estimated our causal effect with our `CausalPy` model. However, we still have to process the resulting data a bit more.

The initial step is to extract the maximum point of our effect, which is typically found at the end of our period. This will enable us to gauge the delta lost by our actions and understand the distance covered by our curve after we reduced advertising spending. We could estimate the daily difference of our effect compared to the previous day and calculate its total, in order to find this number.

In [ ]:
#!collapse
detected_effect = (
    pd.DataFrame(experiment.post_impact.mean(dim=["chain", "draw"]).values) 
    - pd.DataFrame(experiment.post_impact.mean(dim=["chain", "draw"]).values).shift(1)
).sum().item()

print(f"Detected Cumulative effect: {detected_effect:.2f}")

We have observed a total decrease of `-1.33` in the Meta channel post-treatment. This means that we experienced a decline of `1.33` in sales (`Venezuela`) after changing our spend from $X_t$ to $X_{t1}$ amount.

This point is the lost delta contribution over the period from our marketing activities in 'Meta'. However, this delta was caused by the decreased delta in advertising spending in the platform. Therefore, we are going to estimate it by calculating the absolute difference in spending during the period.

In [ ]:
#!collapse
meta_start = df.loc[df["ds"] == start_date, "meta"].values[0]
meta_end = df.loc[df["ds"] == end_date, "meta"].values[0]

delta_x = abs(meta_start - meta_end)

print("Spend before treatment:", meta_start)
print("Spend after treatment:", meta_end)
print("Delta | Change:", delta_x)

This means our negative change of `2.7` units in the `Meta` channel during the intervention is responsible for a decrease of `1.33` sales in *Venezuela*.

> Important: **These two points represent the change on the $X$ and $Y$ axis respectively**.

This is excellent! With this information, we can grasp the incremental impact of our specific action. But **how can we extract even more insights?** We all know that a single event at a specific moment doesn't capture behavior over time. The timing of our analysis can influence the results, so how do we measure this effectively?

*This is precisely why we aim to build a Marketing Mix Model*. We need a methodological framework that enables us to understand incremental effects over various periods. While experiments provide insights into incrementality at a single point in time—often reflecting the average or general behavior of the phenomenon—we need a broader view to capture these dynamics. 

If experiments were our only source of truth, then we wouldn't need a Media Mix Model, would we? Let's think about it: why not just run tests all the time and use those results?

# Extrapolating knowledge from experiments

To extrapolate information from our experiment, we need to understand how our variables are related and how they are generated.

For instance, what is the relationship between our spending and sales? Is it linear or non-linear? What kinds of effects does our spending have? By exploring these questions, we can better grasp the dynamics at play and make more informed estimations.

In this scenario, we will assume that our spending (impressions, clicks, and spend) follows two non-linear functions: adstock (Weibull PDF) and saturation (Michaelis-Menten). Additionally, other factors like holidays and special events are assumed to have linear impacts. This approach helps us understand the complex relationships and influences on our metrics.

This would allow us to use the formula created in the paper [Jin, Yuxue, et al. “Bayesian methods for media mix modeling with carryover and shape effects.” (2017)](https://research.google/pubs/pub46001/) to understand our sales over time.

Now that we have a hypothesis (assumption) about how our independent variable influences our dependent variable, we can evaluate this information to see how it behaves in different situations. This allows us to understand and examine the potential impacts under various conditions.

For example, **our assumption is that marketing effects saturate**, and if that's the case, we can assume causal impact (i.e. the change in Y and change in X) are one coordinate pair in that over-arching non-linear function. Additionally, we assume that said non-linear saturation function is represented by the Michaelis-Menten equation, as mentioned earlier.

Since we are looking at the extra value on the $X$ axis giving an extra value on the $Y$ axis, it is natural to think about estimating the derivative. If we estimate the derivative correctly, we can then obtain the prior values that our function should have to match our experiment.

We will use `sympy` to estimate the derivative of our objective function.

In [ ]:
#!collapse
from sympy import symbols, diff

# Define symbols
x, alpha, lam = symbols("x alpha lambda")

# Define the Michaelis-Menten equation
equation = alpha * x / (lam + x)

# Find its derivative with respect to x
derivative = diff(equation, x)

# Define the derivative of the Michaelis-Menten function based on the sympy expression.
def derivative_michaelis_menten(x, alpha, lam):
    return -alpha * x / (lam + x)**2 + alpha / (lam + x)

Let's remember that the derivative will give us the rate of change on the $Y$ axis associated with the absolute values on $X$. However, since we currently have the absolute values for both variables ($X$ and $Y$), how can we convert this to something useful?

In [ ]:
#!collapse
# Average rate of change as an estimate of the derivative
average_rate_of_change = abs(detected_effect) / delta_x
x_midpoint = (meta_end + meta_start) / 2
print("Average rate of change:", average_rate_of_change)
print("X midpoint:", x_midpoint)

plt.figure(figsize=(10, 5))
plt.scatter(x_midpoint, average_rate_of_change, color="green", label="Experiment", zorder=5)
plt.xlabel("Spend/Impressions (x)")
plt.ylabel("Rate of Change (dy/dx)")
plt.title('Experiment coordinates')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

The previous calculation converts our coordinates to exist in the same two-dimensional space in which our derivative exists, we can construct a function that evaluates the best set of `alpha`, and `lam` values that match the given point.

In [ ]:
#!collapse
# Objective function: Sum of squared differences between observed and predicted dy/dx
from scipy.optimize import minimize
def objective_function(params, x_data, y_data):
    alpha, lam = params
    y_pred = derivative_michaelis_menten(x_data, alpha, lam)
    error = np.sum((y_data - y_pred) ** 2)
    return error

We then optimize this objective using the [`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) function.

In [ ]:
#!collapse
# Single fake data point: (x, dy/dx)
x_range = np.linspace(0, 5, 100)
initial_guess = [1, 1]
experiment_data = np.array([(
    x_midpoint, average_rate_of_change
)])

# Perform optimization with a single point
result_single = minimize(objective_function, initial_guess, args=(experiment_data[:, 0], experiment_data[:, 1]), method='L-BFGS-B', bounds=[(0, None), (0, None)])

# Extract the best alpha and lambda for the single point
best_alpha_single, best_lam_single = result_single.x

# Plot the observed point and the best fit line for the single point
best_fit_y_single = derivative_michaelis_menten(x_range, best_alpha_single, best_lam_single)

print("Best Alpha for Single Point:", best_alpha_single)
print("Best Lambda for Single Point:", best_lam_single)

Great!

We have the best guess `alpha` and `lambda` parameters for our Michaelis-Menten function.

Let's observe the estimated fit for the found `lam` and `alpha`.

In [ ]:
#!collapse
plt.figure(figsize=(10, 5))
plt.plot(x_range, best_fit_y_single, label="Best Fit Derivative", color="grey", linestyle="--")
plt.scatter(experiment_data[:, 0], experiment_data[:, 1], label="Observed Experiment", color="blue")
plt.xlabel("Spend/Impressions (x)")
plt.ylabel("Rate of Change (dy/dx)")
plt.title("Best Fit for Michaelis-Menten Derivative (Single Point)")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

We are getting closer to the interesting part. Let's do a little review of what has happened so far:

1. We created a model in **CausalPy** to estimate the change in our spending in Venezuela after reducing goal spending.
2. We calculated the maximum value of the drop in sales given the drop in advertising spending at goal.
3. We determine how our variables are related, and build a mathematical representation based on those assumptions.
4. We estimate the derivative of the saturation function of our model.
5. We determined our best bet for the parameters of our Michaelis Menten (Saturation) function based on our experiment.

## Let's Model

First, we must separate our regressors and target in two variables.

In [ ]:
#!collapse
X = df[["ds", "meta", "google", "trend"]].copy()
y = df["venezuela"].copy()

We have our prior `lam` and `alpha` points given the experiment, it is time to start modeling. We will create a model in PyMC-Marketing choosing the transformations we want according to our previous assumptions. 

In [ ]:
generic_mmm = MMM(
    date_column = "ds", 
    channel_columns= ["meta","google"], 
    control_columns=["trend"], 
    saturation="michaelis_menten", 
    adstock="weibull_pdf", 
    adstock_max_lag = 4, 
    yearly_seasonality = 4
)

Once we have the model created, let's examine the priors that are given to the model by default.

In [ ]:
#!collapse
generic_mmm.default_model_config

We can notice that alpha and lam are quite wide. Let's quickly visualize using `Preliz` the distributions given for these parameters.

In [ ]:
#!collapse
import preliz as pz

prior_alpha = pz.HalfNormal(sigma=1)
prior_lam = pz.Gamma(mu=2, sigma=1)

print(prior_lam.summary())
prior_lam.plot_pdf()
plt.show()

In [ ]:
#!collapse
print(prior_alpha.summary())
prior_alpha.plot_pdf()
plt.show()

> Note: Let's see that by defining our priors as Gamma or HalfNormal, we are giving structure to our equation, in this case, conditioning those values ​​to be positive.

If we sample the prior parameters given to our generic model using the *sample prior predictive* function, we can then get an idea of our *prior Michaelis Menten curve* and analyze how far/close our experiment is from its mean, given the prior distributions.

In [ ]:
#!collapse
generic_mmm.build_model(X,y)

with generic_mmm.model:
    prior_predictive = pm.sample_prior_predictive(
        random_seed=seed,
        var_names=["saturation_alpha", "saturation_lam"]
    )

prior_alpha_mean = prior_predictive.prior["saturation_alpha"].mean().item()
prior_lambda_mean = prior_predictive.prior["saturation_lam"].mean().item()

x_values = np.linspace(0, 5, 500)
prior_mm_mean_derivative = derivative_michaelis_menten(
    x_values, 
    alpha=(prior_alpha_mean * df.venezuela.max()), 
    lam=(prior_lambda_mean * df.meta.max())
)

plt.figure(figsize=(15, 5))
plt.plot(
    x_values, prior_mm_mean_derivative, 
    color="Blue", label="Prior Median Derivative", linestyle='--'
)

plt.scatter(
    x_midpoint, abs(average_rate_of_change), 
    color="green", label="Experiment", zorder=5
)
plt.xlabel("Spend/Impressions (x)")
plt.ylabel("Rate of Change (dy/dx)")
plt.title("Prior Derivative of Michaelis-Menten Function \n")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

The previous Michaelis-Menten (saturation) curve's derivative, given by the mean of prior distributions for each parameter, is far from our experiment, as expected.

Let's create our own prior distributions based on the estimated `alpha` and `lam` given by the experiment.

*Important*: In order to proceed, it is necessary to scale the values. It's important to remember that **our model internally scales the values using Max Abs Scaler**. This means that the values are divided by their maximum. If `alpha` is on the Y-axis and `lam` is on the X-axis, we need to scale them accordingly before passing them on to the model.

> **Note**: For this reason we multiply the values prior given by the model because these are being using as scaled.

In [ ]:
#!collapse
scaled_alpha_prior = best_alpha_single / df.venezuela.max()
scaled_lam_prior = best_lam_single / df.meta.max()
print(f"Alpha prior Original Scale: {best_alpha_single:.2f}, Lambda prior Original Scale: {best_lam_single:.2f}")
print(f"Alpha prior: {scaled_alpha_prior:.2f}, Lambda prior: {scaled_lam_prior:.2f}")

Now that we are operating in a scaled space, we have noticed that both the values of `alpha` and `lam` lie within a range of 0 to 1. Therefore, we can make use of a Beta distribution, which is limited to positive values and has a range between 0 and 1.

The beta distribution can be characterized by two parameters, namely `alpha` and `beta`. To obtain a distribution with a mean around the scaled saturation values of `lam` and `alpha`, we need to identify a suitable combination of these parameters. This same logic can be applied with any type of distribution. 

You can easily find these values using the PyMC function `find_constrained_prior`. Let's take an example: we want to set the prior for the alpha parameter in Michaelis Menten's function to around 0.48. Using this function, we can look for a set of values that will give us a distribution where most of its mass is between `0.3` and `0.5`.

Let's check the following code:

In [ ]:
#!collapse
alpha_custom_prior = pm.find_constrained_prior(
    pm.Beta, lower=0.3, upper=0.5, init_guess={"alpha": 2, "beta": 3}
)

lam_custom_prior = pm.find_constrained_prior(
    pm.Beta, lower=0.6, upper=0.9, init_guess={"alpha": 1, "beta": 3}
)

print(f"Custom prior parameter for Alpha Saturation (Beta distribution): {alpha_custom_prior}")
print(f"Custom prior parameter for Lambda Saturation (Beta distribution): {lam_custom_prior}")

As usual, let's see the new distributions with Preliz.

In [ ]:
#!collapse
custom_prior_alpha = pz.Beta(**alpha_custom_prior)
custom_prior_lam = pz.Beta(**lam_custom_prior)

custom_prior_alpha.plot_pdf()
custom_prior_lam.plot_pdf()
plt.show()

The new distributions look much better now! Since we know the actual values of the function parameters, it's easy to compare and see how similar the custom and generic prior distributions are to the actual values.

In [ ]:
#!collapse
n_samples = 100
# default prior original scale
default_alpha_prior_orignal_scale = prior_alpha.rvs(n_samples) * df.venezuela.max()
default_lam_prior_orignal_scale = prior_lam.rvs(n_samples) * df.meta.max()
# custom prior original scale
custom_alpha_prior_original_scale = custom_prior_alpha.rvs(n_samples) * df.venezuela.max()
custom_lam_prior_original_scale = custom_prior_lam.rvs(n_samples) * df.meta.max()

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 5), sharex=True, sharey=True)
sns.violinplot(data=default_alpha_prior_orignal_scale, color="blue", orient="h", ax=ax[0][0], label="prior Alpha defult")
sns.violinplot(data=default_lam_prior_orignal_scale, color="blue", orient="h", ax=ax[0][1], label="prior Lam default")

sns.violinplot(data=custom_alpha_prior_original_scale, color="red", orient="h", ax=ax[1][0], label="prior Alpha custom")
sns.violinplot(data=custom_lam_prior_original_scale, color="red", orient="h", ax=ax[1][1], label="prior Lam custom")

# Vertical lines for the true values
ax[0][0].axvline(2.75, color="black", linestyle="--")
ax[0][1].axvline(2.50, color="black", linestyle="--")

ax[1][0].axvline(2.75, color="black", linestyle="--")
ax[1][1].axvline(2.50, color="black", linestyle="--")

ax[0][0].set(title="Alpha Prior (Original Scale)",)
ax[0][1].set(title="Lambda Prior (Original Scale)",)

ax[1][0].set(xlabel="Prior Value")
ax[1][1].set(xlabel="Prior Value")

#add name on the Y axis column row 0
ax[0][0].set(ylabel="Default")
ax[1][0].set(ylabel="Custom")
plt.show()

Our new distributions are now more informative than before. We will pass this information to our model using the **model config** parameter, which receives a dictionary.

In [ ]:
#!collapse
from pymc_marketing.prior import Prior
custom_model_config = {
    "saturation_alpha": Prior(
        "Beta", 
        alpha=np.array([alpha_custom_prior["alpha"], 1.8]), 
        beta=np.array([alpha_custom_prior["beta"], 2]), 
        dims="channel"
    ),
    "saturation_lam": Prior(
        "Beta", 
        alpha=np.array([alpha_custom_prior["alpha"], 1.8]), 
        beta=np.array([alpha_custom_prior["beta"], 2]), 
        dims="channel"
    )
}

Now we have our priors in place to be used by the model.

Even with just one single point of experimentation, our model can now utilize greater knowledge to converge to the actual causal relationship between our target and the marketing actions. 

Let's create a new model!

In [ ]:
mmm = MMM(
    date_column = "ds", 
    channel_columns= ["meta","google"], 
    control_columns=["trend"], 
    saturation="michaelis_menten", 
    adstock="weibull_pdf", 
    adstock_max_lag = 4, 
    yearly_seasonality = 4,
    model_config=custom_model_config
)

Let's take one final look at the custom priors we've derived before we proceed with model training. We can compare our derivative of the Michaelis Menten function with the new priors against the generic ones provided by the model to see how they measure up.

In [ ]:
#!collapse
mmm.build_model(X,y)

with mmm.model:
    custom_prior_predictive = pm.sample_prior_predictive(
        random_seed=seed,
        var_names=["saturation_alpha", "saturation_lam"]
    )

As before we have to sample using the `sample_prior_predictive` function. Now using the new model.

In [ ]:
#!collapse
custom_prior_alpha_mean = custom_prior_predictive.prior["saturation_alpha"].mean().item()
custom_prior_lambda_mean = custom_prior_predictive.prior["saturation_lam"].mean().item()

# x_values = np.linspace(0, 5, 500)
custom_prior_mm_mean_derivative = derivative_michaelis_menten(
    x_values, 
    alpha=(custom_prior_alpha_mean * df.venezuela.max()), 
    lam=(custom_prior_lambda_mean * df.meta.max())
)

plt.figure(figsize=(15, 5))
plt.plot(
    x_values, custom_prior_mm_mean_derivative, 
    color="purple", label="Custom Prior Median Derivative", linestyle="--"
)

plt.plot(
    x_values, prior_mm_mean_derivative, 
    color="blue", label="Prior Median Derivative", linestyle="--"
)

plt.scatter(
    x_midpoint, abs(average_rate_of_change), 
    color="black", label="Experiment", zorder=5
)
plt.xlabel("Spend/Impressions (x)")
plt.ylabel("Rate of Change (dy/dx)")
plt.title("Prior Derivative of Michaelis-Menten Function \n")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

As expected, the new derivative is much closer to our experiment, compared to the generic model. This means that we can proceed with training our Marketing Mix Modeling 🔥

In [ ]:
mmm.fit(X=X, y=y, target_accept=0.98, chains=4, random_seed=rng)

In [ ]:
#!collapse
az.summary(mmm.fit_result, var_names=["saturation_lam", "saturation_alpha"], round_to=2)

The model seems to face some convergence issues related to the `lambda` parameter as it is attempting to fit the data with prior information. Despite these challenges, the recovered data is very close to the actual data.

In [ ]:
#!collapse
print("Posterior Alpha and Lambda values: {:.2f}, {:.2f}".format(
    mmm.idata.posterior["saturation_alpha"].sel(channel="meta").mean().item() * df.venezuela.max(), 
    mmm.idata.posterior["saturation_lam"].sel(channel="meta").mean().item() * df.meta.max()
    )
)

print(
    "True Alpha and Lambda values: {:.2f}, {:.2f}".format(2.75, 2.50)
)

It appears that the estimation of Facebook was more accurate than that of Google (as expected), with the values being more aligned with the reality. If we wanted to go even deeper we could play with Google, set up an experiment and estimate informative priors that help the model find the real values.

Wait for a new article, if you want to see how we make it!

## Final considerations

Please keep in mind that we estimate the values of lambda and alpha with a single experiment. However, it is important to note that our current PyMC models do not consider time as a factor in our regression. Therefore, the contribution estimate is average. 

It is essential to understand that every experiment is inherently connected to the time in which it is executed. That means your model and experiments may not agree entirely. After running multiple experiments, the average contribution detected in total (considering each experiment) should align with the model. However, a single experiment may not. The nature of time variation was ignored in this example to simplify the case. However, in real life, it should be considered.

## Bonus

You can tweak the code mentioned earlier to find the best *alpha* and *lam* values for all the experiments you've run. This will give you more insights than just using a single point to determine the values. Keep in mind that for one point, there can be an endless number of curves that can be adjusted to fit it.

In [ ]:
#!collapse
# fake data point: (x, dy/dx)
x_range = np.linspace(0, 5, 100)
initial_guess = [5, 2]
experiment_data = np.array([
    (x_midpoint, average_rate_of_change), # Experiment 1.
    (1,1), # Experiment 2.
    (.5, 1.5), # Experiment 3.
    (3, .5), # Experiment 4.
    (5, .1), # Experiment 5.
    (.1, 2.5) # Experiment 6.
])

# Perform optimization with a single point
result_single = minimize(objective_function, initial_guess, args=(experiment_data[:, 0], experiment_data[:, 1]), method='L-BFGS-B', bounds=[(0, None), (0, None)])

# Extract the best alpha and lambda for the single point
best_alpha_single, best_lam_single = result_single.x

# Plot the observed point and the best fit line for the single point
best_fit_y_single = derivative_michaelis_menten(x_range, best_alpha_single, best_lam_single)

print("Best Alpha for Single Point:", best_alpha_single)
print("Best Lambda for Single Point:", best_lam_single)

plt.figure(figsize=(12, 5))
plt.plot(x_range, best_fit_y_single, label="Best Fit Derivative", color="grey", linestyle="--")
plt.scatter(experiment_data[:, 0], experiment_data[:, 1], label="Observed Experiments", color="lightgreen")
plt.xlabel("Spend/Impressions (x)")
plt.ylabel("Rate of Change (dy/dx)")
plt.title("Best Fit for Michaelis-Menten Derivative (Multiple Experiments)")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))

plt.show()

We just showed how to come up with useful priors for Marketing Mix Modelings using **PyMC-Marketing** and experiments. By using causal analysis with `CausalPy`, we can figure out "what if" scenarios and estimate how marketing activities contribute to sales. This can help us get better priors in Bayesian MMMs, which means we can decompose more accurately and understand the impact of our marketing efforts better.

We've given you a step-by-step guide to using Bayesian methods and causal inference in your marketing analysis. This should help you make more informed decisions and optimize your marketing strategies more efficiently.

Now it's your turn to put this knowledge into practice and give us your feedback. We're always looking to improve!

**Recommended readings**:
1. [Causal Inference for the Brave and True](https://matheusfacure.github.io/python-causality-handbook/landing-page.html)
2. [What if? Causal inference through counterfactual reasoning in PyMC](https://www.pymc-labs.com/blog-posts/causal-inference-in-pymc/)
3. [Causal analysis with PyMC: Answering "What If?" with the new do operator](https://www.pymc-labs.com/blog-posts/causal-analysis-with-pymc-answering-what-if-with-the-new-do-operator/)

## Speacial Contributors
- Ulf Aslak
- William Dean
- Juan Orduz
- Benjamin J. Vincent
- Thomas Wiecki
---
summary: Learn how to create Bayesian prior for your Marketing Mix Modelinging